In [24]:
# импорт необходимых библиотек
import pandas as pd
import os
from docx import Document
from pony.orm import *
from models import db

In [25]:
# путь к файлам
PACH_DATA = "../raw_data/"

In [26]:
# вывод файлов директории "data"
data_list = os.listdir(PACH_DATA)
data_list

['1_Многоквартирные дома с технико-экономическими характеристиками.xlsx',
 '2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства (2).xlsx',
 '2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства.xlsx',
 '3_Работы по капитальному ремонту, проведенные в многоквартирных домах.xlsx',
 '4_Виды работ по капитальному ремонту многоквартирных домов.xlsx',
 '4_Виды работ по содержанию и ремонту общего имущества многоквартирных домов.xlsx',
 '4_Виды_работ_по_капитальному_ремонту_многоквартирных_домов_с_фактом (3).xlsx',
 '5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx',
 'Объекты_модели_Группы_видов_работ_по_содержанию_и_ремонту_общего.xlsx',
 'Описание примеров данных.docx']

### Описание примеров данных

In [27]:
# загрузка данных файла 'Описание примеров данных.docx'
document = Document(PACH_DATA + data_list[-1])

In [28]:
# парсинг таблиц данных файла 'Описание примеров данных.docx'
df_tables = []
for table in document.tables:
    df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
    for i, row in enumerate(table.rows):
        for j, cell in enumerate(row.cells):
            if cell.text:
                df[i][j] = cell.text
    df_tables.append(pd.DataFrame(df[1:], columns=df[0]))

In [29]:
# количество обнаруженных таблиц
len(df_tables)

16

### 5. Типы событий, регистрируемых по типу объекта многоквартирный дом

In [30]:
# путь к файлу
data_list[7]

'5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx'

In [31]:
# вывод наименование листов в файле "5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx"
xlsx_file = pd.ExcelFile(PACH_DATA + data_list[7])
sheet_names = xlsx_file.sheet_names
print(sheet_names)

['Лист1']


In [32]:
# загрузка данных листа "Лист1" в файле "5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx"
df_systems = pd.read_excel(PACH_DATA + data_list[7], sheet_name="Лист1")
df_systems.head()

,id,name,system
0,001b91e9-2ac5-4949-bff0-bb51d4f3057c,Нарушение в работе АГВ,MOS_GAS
1,0089fd48-aa7a-443d-85a5-00470285b076,Проверить на загазованность дюкер Гольяновский,MOS_GAS
2,016697a2-feef-4568-9629-3c9dddae3aea,Разрушение асфальтового полотна вокруг ковера,MOS_GAS
3,02212d7e-3c4c-4e6a-8f6b-d4f8f1b82e72,Угроза взрыва; Запах газа в доме,MOS_GAS
4,0250d455-608a-4d63-9902-ebe3b82d0b67,Запах газа от ГРП,MOS_GAS


In [33]:
# размерность данных
df_systems.shape

(950, 3)

In [34]:
# признаки данных
df_systems.columns

Index(['id', 'name', 'system'], dtype='object')

In [35]:
# описание Типы событий, регистрируемых по типу объекта-многоквартирный дом
df_tables[15]

,Имя колонки,Комментарий
0,id,Идентификатор
1,name,Наименование типа события
2,system,Информационная система – источник события


In [36]:
# проверка на наличие пропусков
df_systems.isna().sum()

id        0
name      1
system    0
dtype: int64

In [37]:
# удаление пропусков
df_systems = df_systems.dropna()

In [38]:
# количество уникальных значений
df_systems.nunique()

id        949
name      791
system     10
dtype: int64

В данных присутствуют **дубликаты** \
В том числе присутствуют **дубликаты** наименования типа события по разным информационным системам. \
Удалять пока **не** будем 

In [39]:
# удаление парных дубликатов
df_systems = df_systems.drop_duplicates(subset=["name", "system"])

In [40]:
# количество наименований типа событий по информационным системам
df_systems["system"].value_counts()

system
MOS_GAS    555
MVK        116
NG          55
EDC         54
ASUPR       42
EVAGD       38
MGI         34
KGH          3
GORMOST      3
CAFAP        3
Name: count, dtype: int64

In [41]:
# вывод таблицы 'systems'
db.System.select().show()

id|system_name
--+-----------


In [42]:
@db_session
def add_system(system_name):

    """ функуия записи наименования информационной системы 
    - источник события в таблицу 'systems' """
    
    db.System(system_name=system_name)
    # commit() will be done automatically
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

In [43]:
# данные для записи в таблицу
system_names = list(df_systems["system"].unique())
system_names[:2]

['MOS_GAS', 'MVK']

In [44]:
# запись данных в базу данных
for system_name in system_names:
    add_system(system_name)

CacheIndexError: Cannot create System: value 'MOS_GAS' for key system_name already exists

In [ ]:
# проверка записи данных
db.System.select()[:10].show()

id|system_name
--+-----------
1 |MOS_GAS    
2 |MVK        
3 |EDC        
4 |ASUPR      
5 |MGI        
6 |NG         
7 |EVAGD      
8 |KGH        
9 |GORMOST    
10|CAFAP      


In [ ]:
# проверка записи данных
db.select("select count(*) from systems")[0], len(system_names)

(10, 10)

Данные записаны **успешно**